In [6]:
pip install sympy

Note: you may need to restart the kernel to use updated packages.


In [64]:
# Tables for subkey generation (k1, k2, k3, ... k10)
P10 = (3, 5, 2, 7, 4, 10, 1, 9, 8, 6)
P8 = (6, 3, 7, 4, 8, 5, 10, 9)
P4 = (2, 4, 3, 1)

# Tables for initial and final permutations (b1, b2, b3, ... b8)
IP = (2, 6, 3, 1, 4, 8, 5, 7)
IPi = (4, 1, 3, 5, 7, 2, 8, 6)

# Tables for the fk function
E = (4, 1, 2, 3, 2, 3, 4, 1)
S0 = [
        [1, 0, 3, 2],
        [3, 2, 1, 0],
        [0, 2, 1, 3],
        [3, 1, 3, 2]
     ]
S1 = [
        [0, 1, 2, 3],
        [2, 0, 1, 3],
        [3, 0, 1, 0],
        [2, 1, 0, 3]
     ]

def permutation(pattern, key):
    permuted = ""

    for i in pattern:
        permuted += key[i-1]

    return permuted

def generate_first(left, right):
    left = left[1:] + left[:1]
    right = right[1:] + right[:1]
    key = left + right

    return permutation(P8, key)

def generate_second(left, right):
    left = left[3:] + left[:3]
    right = right[3:] + right[:3]
    key = left + right

    return permutation(P8, key)

def transform(right, key):
    extended = permutation(E, right)
    xor_cipher = bin(int(extended, 2) ^ int(key, 2))[2:].zfill(8)
    xor_left = xor_cipher[:4]
    xor_right = xor_cipher[4:]

    new_left = Sbox(xor_left, S0)
    new_right = Sbox(xor_right, S1)

    return permutation(P4, new_left + new_right)

def Sbox(data, box):
    row = int(data[0] + data[3], 2)
    column = int(data[1] + data[2], 2)

    return bin(box[row][column])[2:].zfill(4)

def encrypt(left, right, key):
    cipher = int(left, 2) ^ int(transform(right, key), 2)

    return right, bin(cipher)[2:].zfill(4)

key = input("Enter a 10-bit key: ")
if len(key) != 10:
    raise Exception("Check the input")

plaintext = input("Enter 8-bit plaintext: ")
if len(plaintext) != 8:
    raise Exception("Check the input")

p10key = permutation(P10, key)
print("First Permutation")
print(p10key)

left_key = p10key[:len(p10key)//2]
print("Left key",left_key)

right_key = p10key[len(p10key)//2:]
print("Right key",right_key)

first_key = generate_first(left_key, right_key)
print("*****")
print("First key")
print(first_key)

second_key = generate_second(left_key, right_key)
print("*****")
print("Second key")
print(second_key)

initial_permutation = permutation(IP, plaintext)
print("Initial Permutation",initial_permutation)

left_data = initial_permutation[:len(initial_permutation)//2]
right_data = initial_permutation[len(initial_permutation)//2:]

left, right = encrypt(left_data, right_data, first_key)
left, right = encrypt(left, right, second_key)

print("Ciphertext:", permutation(IPi, left + right))

Enter a 10-bit key: 1010110101
Enter 8-bit plaintext: 01010101
First Permutation
1100011011
Left key 11000
Right key 11011
*****
First key
10001111
*****
Second key
11111001
Initial Permutation 11001100
Ciphertext: 01111001


In [65]:
# 2 hellman

from random import randint

if __name__ == '__main__':

	# Both the persons will be agreed upon the
	# public keys G and P
	# A prime number P is taken
	P = 23
	
	# A primitve root for P, G is taken
	G = 9
	
	
	print('The Value of P is :%d'%(P))
	print('The Value of G is :%d'%(G))
	
	# Alice will choose the private key a
	a = 4
	print('The Private Key a for Alice is :%d'%(a))
	
	# gets the generated key
	x = int(pow(G,a,P))
	
	# Bob will choose the private key b
	b = 3
	print('The Private Key b for Bob is :%d'%(b))
	
	# gets the generated key
	y = int(pow(G,b,P))
	
	
	# Secret key for Alice
	ka = int(pow(y,a,P))
	
	# Secret key for Bob
	kb = int(pow(x,b,P))
	
	print('Secret key for the Alice is : %d'%(ka))
	print('Secret Key for the Bob is : %d'%(kb))

The Value of P is :23
The Value of G is :9
The Private Key a for Alice is :4
The Private Key b for Bob is :3
Secret key for the Alice is : 9
Secret Key for the Bob is : 9


In [68]:
#4 RSA
import random


'''
Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers
'''

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


'''
Euclid's extended algorithm for finding the multiplicative inverse of two numbers
'''


def multiplicative_inverse(e, phi):
    d = 0
    x1 = 0
    x2 = 1
    y1 = 1
    temp_phi = phi

    while e > 0:
        temp1 = temp_phi//e
        temp2 = temp_phi - temp1 * e
        temp_phi = e
        e = temp2

        x = x2 - temp1 * x1
        y = d - temp1 * y1

        x2 = x1
        x1 = x
        d = y1
        y1 = y

    if temp_phi == 1:
        return d + phi


'''
Tests to see if a number is prime.
'''


def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True


def generate_key_pair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    # n = pq
    n = p * q

    # Phi is the totient of n
    phi = (p-1) * (q-1)

    # Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    # Use Euclid's Algorithm to verify that e and phi(n) are coprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    # Use Extended Euclid's Algorithm to generate the private key
    d = multiplicative_inverse(e, phi)

    # Return public and private key_pair
    # Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))


def encrypt(pk, plaintext):
    # Unpack the key into it's components
    key, n = pk
    # Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [pow(ord(char), key, n) for char in plaintext]
    # Return the array of bytes
    return cipher


def decrypt(pk, ciphertext):
    # Unpack the key into its components
    key, n = pk
    # Generate the plaintext based on the ciphertext and key using a^b mod m
    aux = [str(pow(char, key, n)) for char in ciphertext]
    # Return the array of bytes as a string
    plain = [chr(int(char2)) for char2 in aux]
    return ''.join(plain)


if __name__ == '__main__':
    '''
    Detect if the script is being run directly by the user
    '''
    print("RSA")

    p = int(input(" - Enter a prime number (17, 19, 23, etc): "))
    q = int(input(" - Enter another prime number"))

    print(" - Generating your public / private key-pairs now . . .")

    public, private = generate_key_pair(p, q)

    print(" - Your public key is ", public, " and your private key is ", private)

    message = input(" - Enter a message to encrypt with your public key: ")
    encrypted_msg = encrypt(public, message)

    print(" - Your encrypted message is: ", ''.join(map(lambda x: str(x), encrypted_msg)))
    print(" - Decrypting message with private key ", private)
    print(" - Your message is: ", decrypt(private, encrypted_msg))

RSA
 - Enter a prime number (17, 19, 23, etc): 17
 - Enter another prime number23
 - Generating your public / private key-pairs now . . .
 - Your public key is  (31, 391)  and your private key is  (511, 391)
 - Enter a message to encrypt with your public key: ffgfgd
 - Your encrypted message is:  204204341204341331
 - Decrypting message with private key  (511, 391)
 - Your message is:  ffgfgd


In [71]:
'''
Public Key is (e,n)
Private Key is (d,n)
'''
def generate_keys():

  from random import randint
  from math import gcd
  from sympy import mod_inverse

  # Generate Two unequal Large Primes of comparable size
  p, q = 877, 751
  #p, q = 6971, 6299

  # For large p and q, n will take centuries to factorize
  n = p*q

  # phi function
  fi_n = (p-1)*(q-1)
  
  # e and fi_n are relatively prime if their gcd is 1
  while True:
    e = randint(1, fi_n)
    if gcd(fi_n, e) == 1:
      break
  
  # inverse modulo exists iff e and fi_n are relatively prime
  # Modular Inverse
  d = mod_inverse(e, fi_n)

  return (e,n), (d,n)


public_key, private_key = generate_keys()

In [72]:
def encryption(m, public_key):
  e, n = public_key
  c = m**e % n
  return c

c = encryption(ord('A'), public_key)
c

277089

In [73]:
def decryption(c, private_key):
  d, n = private_key
  p = c**d % n
  return p

p = decryption(c, private_key)
chr(p)


'A'

In [74]:
def encrypt_text(plain_text, public_key):
  cipher_text = ''
  for character in plain_text:
    cipher_text += chr(encryption(ord(character), public_key))
  return cipher_text

cipher_text = encrypt_text('dsdsds', public_key)
cipher_text

'\U00014b87𢷲\U00014b87𢷲\U00014b87𢷲'

In [75]:
def decrypt_text(cipher_text, private_key):
  decrypted_text = ''
  for character in cipher_text:
    decrypted_text += chr(encryption(ord(character), private_key))
  return decrypted_text

decrypted_text = decrypt_text(cipher_text, private_key)
decrypted_text

'dsdsds'

In [14]:
# 5 ECC
import math
import random

def point(a, b):

	if (4*(a**3) + 27*(b**2)) != 0:
		x = 1
		print("generating")
		while True:
			rhs = (x**3) + (a*x) + b
			y = int(math.sqrt(rhs))
			lhs = (y**2)

			if lhs == rhs:
				return [x, y]
			else:
				x += 1
	else:
		print("Enter another coefficients.")


a = int(input("Enter the coefficient 'a' of curve: "))
b = int(input("Enter the coefficient 'b' of curve: "))

private_A = 13
private_B = 15

generator = point(a, b)
print("Generator point: ", generator)

m = int(input("Enter the plaintext integer: "))

public_key_A = [private_A*generator[0], private_A*generator[1]]
print("Public Key of A: ", public_key_A)

public_key_B = [private_B*generator[0], private_B*generator[1]]
print("Public Key of B: ", public_key_B)

k = random.randint(0, 10)

c1 = k * (generator[0] + generator[1])

c2 = m + ((k*public_key_B[0]) + (k*public_key_B[1]))

ciphertext = [c1, c2]
print("Ciphertext: ", ciphertext)

r = private_B*c1

plaintext = c2 - r
print("Decrypted Plaintext: ", plaintext)


Enter the coefficient 'a' of curve: 3
Enter the coefficient 'b' of curve: 4
generating
Generator point:  [5, 12]
Enter the plaintext integer: 1010101010
Public Key of A:  [65, 156]
Public Key of B:  [75, 180]
Ciphertext:  [170, 1010103560]
Decrypted Plaintext:  1010101010
